In [4]:
# Batch size = 4 as used in 4.1_LR(0.001)_sgd.ipynb 
# Epochs = 20 
# Optimizer = SGD
# Learning rate = 0.001
# This is in comparison to that of 4.1_LR(0.001)_sgd.ipynb

import os
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["MKL_NUM_THREADS"] = "4"

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [5]:
traindata = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(traindata, batch_size=4, shuffle=True, num_workers=2)

testdata = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testdata, batch_size=4, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.fc1 = nn.Linear(64 * 3 * 3, 64)
        self.fc2 = nn.Linear(64, 10)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = x.view(-1, 64 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()

criterion = nn.KLDivLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

for epoch in range(20):
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        labels = torch.nn.functional.one_hot(labels, num_classes=10)

        optimizer.zero_grad()
        outputs = net(inputs)
        # outputs = F.log_softmax(outputs, dim=1)
        # loss = criterion(outputs, labels)
        outputs = torch.nn.functional.log_softmax(outputs)
        loss = criterion(outputs.float(), labels.float())

        loss.backward()
        optimizer.step()



/tmp/ipykernel_256142/2330911602.py:40: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = torch.nn.functional.log_softmax(outputs)


In [10]:
# Give training accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        inputs, labels = data
        labels = torch.nn.functional.one_hot(labels, num_classes=10)
        outputs = net(inputs)
        outputs = torch.nn.functional.log_softmax(outputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        # print(predicted, labels)
        correct += (predicted == labels).sum().item()
        

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        labels = torch.nn.functional.one_hot(labels, num_classes=10)
        outputs = net(inputs)
        outputs = torch.nn.functional.log_softmax(outputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy: %d %%' % (100 * correct / total))

/tmp/ipykernel_256142/4235028227.py:9: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = torch.nn.functional.log_softmax(outputs)


RuntimeError: The size of tensor a (4) must match the size of tensor b (10) at non-singleton dimension 1

In [11]:
# Class wise accuracy

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        labels = torch.nn.functional.one_hot(labels, num_classes=10)
        outputs = net(inputs)
        outputs = torch.nn.functional.log_softmax(outputs)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

/tmp/ipykernel_256142/650603460.py:10: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = torch.nn.functional.log_softmax(outputs)


RuntimeError: The size of tensor a (4) must match the size of tensor b (10) at non-singleton dimension 1